In [2]:
import os
import timeit
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score

In [3]:
import_path = os.path.join(os.getcwd(), 'saved_models', 'german_sign_classifier_model')
converter = tf.lite.TFLiteConverter.from_saved_model(import_path)
tflite_model = converter.convert()

os.makedirs(os.path.join(os.getcwd(), 'saved_tflite_models'), exist_ok=True)

tflite_model_path = os.path.join(os.getcwd(), 'saved_tflite_models', 'german_signs_classifier.tflite')
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [4]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the CONV_2D op takes
tensor #0 and tensor #1 and tensor #2 as input and produces tensor #195 as output.

Subgraph#0 main(T#0) -> [T#321]
  Op#0 CONV_2D(T#0, T#1[], T#2[]) -> [T#195]
  Op#1 CONV_2D(T#195, T#98[], T#3[]) -> [T#196]
  Op#2 CONV_2D(T#196, T#99[], T#4[]) -> [T#197]
  Op#3 MAX_POOL_2D(T#197) -> [T#198]
  Op#4 CONV_2D(T#198, T#100[], T#5[]) -> [T#199]
  Op#5 CONV_2D(T#199, T#101[], T#6[]) -> [T#200]
  Op#6 MAX_POOL_2D(T#200) -> [T#201]
  Op#7 AVERAGE_POOL_2D(T#201) -> [T#202]
  Op#8 CONV_2D(T#201, T#102[], T#7[]) -> [T#203]
  Op#9 CONV_2D(T#201, T#103[], T#8[]) -> [T#204]
  Op#10 CONV_2D(T#204, T#104[], T#9[]) -> [T#205]
  Op#11 CONV_2D(T#201, T#105[], T#10[]) -> [T#206]
  Op#12 CONV_2D(T#206, T#106[], T#11[]) -> [T#207]
  Op#13 CONV_2D(T#207, T#107[], T#12[]) -> [T#208]
  Op#14 CONV_2D(T#202, T#108[], T#13[]) ->

In [14]:
def getfoldersize(path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    total_size /= (1024 ** 3)
    total_size = round(total_size, 3)
    return total_size

old_model_size = getfoldersize(os.path.join(os.getcwd(),'saved_models','german_sign_classifier_model'))
new_model_size = getfoldersize(os.path.join(os.getcwd(),'saved_tflite_models'))

print(old_model_size, new_model_size) # 60-70% smaller 

0.274 0.089


In [15]:
data_dir = './data/archive/Train/'
IMAGE_SHAPE = (299, 299)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SHAPE,
    batch_size=1)

Found 39209 files belonging to 43 classes.
Using 7841 files for validation.


In [16]:
for image, label in val_ds.take(1):
    start_time = timeit.default_timer()
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    input_shape = input_details[0]['shape']
    input_data = np.asarray(image, dtype=np.float32)
    
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    elapsed_time = timeit.default_timer() - start_time
    print(elapsed_time) # seconds I think so a little over 1/3rd of a second

0.2396446999919135


In [17]:
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
true = []
pred = []
for image, label in val_ds:
    input_data = np.asarray(image, dtype=np.float32)
    
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    true.append(label.numpy()[0])
    pred.append(np.argmax(output_data))
    
accuracy = accuracy_score(true, pred)
print(accuracy) # same score

0.9959188878969519
